# N° projet - Projet - Note de calcul
---
## Element : Assemblage bois/métal
- Annexe : ANxx
- Date : 13/11/2022
- Ingénieur : Anthony PARISOT
---

In [1]:
# Pour impression sans le code executer cette commande dans un terminale:
#               jupyter nbconvert --no-input --to html Nom_fichier.ipynb    

In [2]:
from math import *

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import handcalcs.render
from handcalcs.decorator import handcalc
import forallpeople as si

from eurocode import EC5_Element_droit as EC5_Ele
from eurocode import EC5_Assemblage as EC5_Ass
from eurocode import EC3_Assemblage as EC3_Ass

ModuleNotFoundError: No module named 'numpy'

In [3]:
si.environment("structural")
si.environment()


{'mm': 1.000 mm, 'ft': 1.000 ft, 'inch': 1.000 inch, 'N': 1.000 N, 'kN': 1.000 kN, 'MN': 1.000 MN, 'Pa': 1.000 Pa, 'kPa': 1.000 kPa, 'MPa': 1.000 MPa, 'GPa': 1.000 GPa, 'Nm': 1.000 N·m, 'N_m': 1.000 N/m, 'lbft': 1.000 lb·ft, 'kipft': 1.000 kip·ft, 'psf': 1.000 psf, 'lbin': 1.000 lb·in, 'kipin': 1.000 kip·in, 'ksf': 1.000 ksf, 'lb': 1.000 lb, 'lb_ft': 1.000 lb/ft, 'kip_ft': 1.000 kip/ft, 'kip': 1.000 kip, 'lbm': 1.000 lbm, 'psi': 1.000 psi, 'ksi': 1.000 ksi, 'pcf': 1.000 pcf, 'kcf': 1.000 kcf, 'pci': 1.000 pci, 'kci': 1.000 kcf, 'N_m3': 1.000 N/m³, 'Nm2': 1.000 N·m², 'lbft2': 1.000 lb·ft², 'kft2': 1.000 kip·ft²} 
 {'kg': 1.000 kg, 'm': 1.000 m, 's': 1.000 s, 'A': 1.000 A, 'cd': 1.000 cd, 'K': 1.000 K, 'mol': 1.000 mol}


### Définition des éléments
---

#### Générale

In [4]:
%%render params 1
classe_service = 2

<IPython.core.display.Latex object>

#### Bois

In [5]:
%%render params 1

classe = "C24"
b1 = 60*si.mm
h1 = 140*si.mm
Hi = 12 # Humidité initiale en pourcent

<IPython.core.display.Latex object>

- <font color="red">**Caratéristique du bois**</font>

In [6]:
Beam = EC5_Ele.Beam(b1, h1, Hi= Hi, classe= classe, cs= classe_service)
Beam.caract_meca

fm0k          24
ft0k        14.5
ft90k        0.4
fc0k          21
fc90k        2.5
fvk            4
E0mean     11000
E005        7400
E90mean      370
Gmoy         690
rhok         350
rhomean      420
Name: C24, dtype: object

- <font color="red">**Kmod**</font>

In [7]:
Beam.kmod

,CS,Permanente,Long terme,Moyen terme,Court terme,Instantanee
Materiau,,,,,,
Massif,2,0.6,0.7,0.8,0.9,1.1


In [8]:
%%render params

k_mod = Beam.kmod["Moyen terme"].iloc[0]

<IPython.core.display.Latex object>

- <font color="red">**Gamma M**</font>

In [9]:
%%render params
gamma_m_bois_fond = Beam.gamma_M[0] # Fondamentales
gamma_m_bois_acc = Beam.gamma_M[1] # Accidentelles

<IPython.core.display.Latex object>

#### Plaque acier

In [10]:
%%render params 1
classe_acier = "S275"
ep_plaque = 6*si.mm

<IPython.core.display.Latex object>

#### Boulon

In [11]:
%%render params 1

qualite_acier = 8.8
d = 12*si.mm
d_0 = 14*si.mm 

<IPython.core.display.Latex object>

In [12]:
boulon = EC3_Ass.Boulon(int(d), int(d_0), qualite_acier, verif_filetage=False, filetage_EN1090=True, t=int(ep_plaque), classe=classe_acier)

- **Caractéristique de la plaque**

In [13]:
%%render params 0
f_y = boulon.fy * si.MPa
f_u = boulon.fu * si.MPa

<IPython.core.display.Latex object>

- **Caractéristique du boulon**

In [14]:
%%render params 0
f_yb = boulon.fyb * si.MPa
f_ub = boulon.fub * si.MPa

<IPython.core.display.Latex object>

- **Pince acier selon EN 1993 1-8**

In [15]:
boulon.pince_metal_boulon(trous_oblongs=False, corrosion=False, EN_10025_5=True)

{'e1': {'e1_min': 16.8, 'e1_max': 125},
 'e2': {'e2_min': 16.8, 'e2_max': 125},
 'p1': {'p1_min': 30.8, 'p1_max': 84},
 'p2': {'p2_min': 33.6, 'p2_max': 84}}

> Pinces considérées

In [16]:
%%render params 0

e_1 = 20*si.mm
e_2 = 20*si.mm
p_1 = 60*si.mm
p_2 = 40*si.mm

<IPython.core.display.Latex object>

### Dimensionnement

#### Effort

In [17]:
%%render params 1
N_Ed = 15*si.kN
V_Ed = 5*si.kN

<IPython.core.display.Latex object>

#### Vérification du boulon

- **Vérification cisaillement, traction et effort combiné**

In [18]:
%%render params 1
A_s = boulon.As * si.mm**2
A_n = boulon.An * si.mm**2
gamma_M2 = boulon.GAMMA_M["gamma_M2"]

<IPython.core.display.Latex object>

In [19]:
%%render 1
F_tRd_bl = 0.9 * f_ub * A_s / gamma_M2
F_vRd_bl = 0.6 * f_ub * A_n / gamma_M2

<IPython.core.display.Latex object>

- Taux de tavail
---

In [20]:
verif = "Critère non vérifié"

In [21]:
%%render

if N_Ed <= F_tRd_bl:
    taux = N_Ed / F_tRd_bl

if V_Ed <= F_vRd_bl:
    taux = V_Ed / F_vRd_bl

if V_Ed <= F_vRd_bl + N_Ed / (1.4 * F_tRd_bl) :
    taux = V_Ed / F_vRd_bl + N_Ed / (1.4 * F_tRd_bl) 


<IPython.core.display.Latex object>

- Vérification de la pression diamétrale

In [22]:
%%render params

F_bRd = boulon.FbRd(int(e_1), int(e_2), int(p_1), int(p_2)) * si.N

<IPython.core.display.Latex object>

In [23]:
%%render

if V_Ed <= F_bRd:
    taux = V_Ed / F_bRd

<IPython.core.display.Latex object>